In [6]:
import numpy as np
from abc import ABC, abstractmethod

In [7]:
class Predictor(ABC):
    def __init__(self, model_filename):
        self.model_filename = model_filename
        self.model = None
        self.model = self.loadModel()

    @abstractmethod
    def loadModel(self):
        pass

    @abstractmethod
    def predict(self, inputs, n_to_predict):
        pass

In [8]:
class XGBoostPredictor(Predictor):
    def loadModel(self):
        
        import xgboost as xgb

        model = xgb.Booster()
        model.load_model(self.model_filename)
        return model
    
    def predict(self, inputs, n_to_predict):

        import xgboost as xgb
        
        from sklearn.preprocessing import MinMaxScaler
        scaler=MinMaxScaler(feature_range=(0,1))
        
        inputs.drop("timestamp",axis=1,inplace=True)
        scaler.fit_transform(inputs.values)
        
        inputs=inputs[len(inputs)-n_to_predict:].values
        inputs=inputs.reshape(-1,1)
        inputs=scaler.transform(inputs)

        d_test = xgb.DMatrix(inputs)
        closing_price=self.model.predict(d_test)
        closing_price=closing_price.reshape(-1, 1)
        closing_price=scaler.inverse_transform(closing_price)
        
        return closing_price

In [9]:
from tensorflow.keras.models import load_model

class RNNPredictor(Predictor):
    def loadModel(self):
        return load_model(self.model_filename)
    
    def predict(self, inputs, n_to_predict):
        inputs.index=inputs.timestamp
        inputs.drop("timestamp",axis=1,inplace=True)
        
        from sklearn.preprocessing import MinMaxScaler
        scaler=MinMaxScaler(feature_range=(0,1))
        scaled_inputs = scaler.fit_transform(inputs.values)
        
        scaled_inputs=scaled_inputs[len(inputs)-n_to_predict:]
        scaled_inputs=scaled_inputs.reshape(-1,1)

        X_test=np.array(scaled_inputs)

        X_test=np.reshape(X_test,(X_test.shape[0],X_test.shape[1],1))
        closing_price=self.model.predict(X_test)
        closing_price=scaler.inverse_transform(closing_price)
        
        return closing_price

In [10]:
class LSTMPredictor(Predictor):
    def loadModel(self):    
        return load_model(self.model_filename)
    
    def predict(self, inputs, n_to_predict):
        
        from sklearn.preprocessing import MinMaxScaler
        scaler=MinMaxScaler(feature_range=(0,1))
        
        inputs.drop("timestamp",axis=1,inplace=True)
        scaler.fit_transform(inputs.values)
        
        inputs=inputs[len(inputs)-n_to_predict:].values
        inputs=inputs.reshape(-1,1)
        inputs=scaler.transform(inputs)

        X_test=np.array(inputs)

        X_test=np.reshape(X_test,(X_test.shape[0],X_test.shape[1],1))
        closing_price=self.model.predict(X_test)
        closing_price=scaler.inverse_transform(closing_price)
        
        return closing_price